# Análisis inferencial

Este notebook busca crear un análisis inferencial de los datos explorados en la etapa 2. Se busca responde a las siguientes preguntas mediante a la definición de intervalos de confianza y pruebas de hipótesis:

- ¿Qué relación existe entre el nivel educativo y el hábito de lectura de libros en México?
- ¿Cómo influye la disponibilidad de libros en el hogar durante la infancia en los hábitos de lectura en la adultez?

## Definición de valores iniciales

Para el análisis inferencial debemos definir las hipótesis nula y alternativa, así como los valores iniciales que nos permitan realizar las pruebas de hipótesis y calcular los intervalos de confianza.

### Pregunta 1

- **Hipótesis nula (H0)**: Las proporciones de personas que leen son iguales en los distintos niveles educativos.
- **Hipótesis alternativa (H1)**: Al menos una proporción es diferente entre los niveles educativos.

Las proporciones de lectura en los distintos niveles educativos son:
- P_bajo: 0, 1, 2, 3
- P_medio: 4, 5, 6
- P_alto: 7, 8, 9

Las variables que se utilizarán para el análisis son:
- `nivel`: Nivel educativo (1 a 9)
- `p3_1`: Se leyeron libros en los últimos 12 meses
- `p4`: Número de libros leídos en los últimos 12 meses

## Pregunta 2

- **Hipótesis nula (H0)**: Las proporciones de lectura en adultez son iguales entre los niveles de disponibilidad de libros en la infancia.
- **Hipótesis alternativa (H1)**: Al menos una proporción es diferente entre los niveles de disponibilidad de libros en la infancia.

Las proporciones de lectura en los distintos niveles de disponibilidad de libros en la infancia son:
- P_bajo: 0, 1
- P_medio: 2, 3
- P_alto: 4, 5, 6

Las variables que se utilizarán para el análisis son:
- `p34_4`: Disponibilidad de libros en la infancia
- `p34_4_1`: Cantidad de libros de la variante anterior
- `p3_1`: Se leyeron libros en los últimos 12 meses